# Langchain
This repository contains examples of how to use Langchain features. Each example demonstrates a different use case, showcasing the flexibility and power of Langchain.

## Create LLM instance
First, we need an instance of the LLM. Here, we are using Gemini as a LLM instance and it will be used to process our prompts.

In [1]:
# Import necessary libraries
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [ ]:
# Import GOOGLE_API_KEY to use gemini model
# https://console.cloud.google.com/apis/credentials
if os.getenv("GOOGLE_API_KEY") is None:
    os.environ["GOOGLE_API_KEY"] = "<YOUR_GOOGLE_API_KEY>"

In [3]:
# Create the LLM instance
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.1,
    max_output_tokens=1024,
)

### Invoke the LLM with single prompt

In [4]:
llm.invoke("Who are you?")  # Example usage

AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--7b656961-9bdc-4a75-b5b4-f8cff02850c4-0', usage_metadata={'input_tokens': 5, 'output_tokens': 11, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})

## Invoke the LLM with multiple messages
We define a list of messages to send to the LLM. Each message can be of different types, such as user messages or system messages.
- HumanMessage: This is a message from the user. Use this to provide input or ask questions as a User.
- SystemMessage: This is a message from the system. Use this to set the context or instructions for the LLM.
- AIMessage: This is a message from the AI. Use this to provide responses or outputs from the LLM.


In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant in a North-Indian Restaurant."),
    HumanMessage(content="What dishes you have in the menu?"),
]

llm.invoke(messages)

AIMessage(content="Namaste! Welcome to our restaurant! We have a wide variety of delicious North Indian dishes to tantalize your taste buds. Here's a glimpse of what we offer:\n\n**Appetizers (Starters):**\n\n*   **Paneer Tikka:** Cubes of marinated paneer (Indian cottage cheese) grilled to perfection with onions and bell peppers.\n*   **Chicken Tikka:** Tender pieces of chicken marinated in yogurt and spices, then grilled.\n*   **Samosa:** Crispy fried pastries filled with spiced potatoes and peas.\n*   **Aloo Tikki:** Spiced potato patties, shallow-fried until golden brown.\n*   **Onion Bhaji:** Finely sliced onions coated in a spiced chickpea flour batter and deep-fried.\n*   **Vegetable Pakora:** Assorted mixed vegetables coated in a spiced chickpea flour batter and deep-fried.\n*   **Hara Bhara Kabab:** Patties made with spinach, peas, and potatoes, pan-fried.\n\n**Main Courses (Curries & Gravies):**\n\n**Vegetarian Delights:**\n\n*   **Paneer Butter Masala:** A rich and creamy to

## Prompt Templates
When you want to use a predefined prompt structure, with placeholders for dynamic content, you can use prompt templates. This allows you to create a consistent format for your prompts while still being able to customize the content.

In [6]:
from langchain_core.prompts import PromptTemplate

system_prompt_template = PromptTemplate(
    input_variables=["cuisine"],
    template="You are a helpful assistant in a {cuisine} restaurant.",
)

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content=system_prompt_template.format(cuisine="Italian")),
    SystemMessage(content="You have pasta, pizza, and risotto. We make best margerita pizza in town!"),
    HumanMessage(content="What dishes you have in the menu?"),
]

llm.invoke(messages)


AIMessage(content="Welcome! We have a delicious selection of Italian favorites for you today. Our specialties include:\n\n*   **Pasta:** We have a variety of classic pasta dishes.\n*   **Pizza:** Of course, we have pizza! And I must say, we make the **best Margherita pizza in town!**\n*   **Risotto:** For something creamy and comforting, we also offer risotto.\n\nIs there anything specific you're in the mood for, or would you like me to tell you more about any of these categories?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--9a79e6f4-dc1e-41d4-8413-642f7bd73f86-0', usage_metadata={'input_tokens': 38, 'output_tokens': 108, 'total_tokens': 146, 'input_token_details': {'cache_read': 0}})

## Chains
Chains are useful when you want to combine two or more actions which create a workflow. Chains allow user to link multiple components together, passing the output of one component as the input to another. This is useful for building more sophisticated applications that require multiple steps or components.

### SimpleSequentialChain
SimpleSequentialChain is a type of chain that allows you to execute multiple chains in sequence. The output of one chain becomes the input for the next chain in the sequence. Key feature is, it only accepts a single input variable for all the chains in the sequence. This is useful when you want to create a workflow where the output of one step is directly used as the input for the next step.

In [7]:

# Import necessary components for prompt templates and chains
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
# type_of_rest: This instance will be used to generate a menu and specialty dish based on the cuisine type.
type_of_rest = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["cuisine_type"],
        template="You are a helpful assistant in a {cuisine_type} restaurant. When asked about the menu, list 5 names of the dishes and 1 specialty dish with ingredients.",
    ),
    output_key="ingredients",
)

# list_alergens: This instance will be used to analyze the specialty dish for allergens.
list_alergens = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["dish",],
        template="You are a food quality analyst. You test food allergens and quality when asked about the speciality dish. Provide list of names of allergens into given dish. No any additional information, just list the allergens. \n Dish Details: {dish}",
    ),
)

/var/folders/vd/bybb603x54vf1kyhghxkssbc0000gn/T/ipykernel_48054/38766143.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  type_of_rest = LLMChain(


In [9]:
# Create a simple chain and inject input for the type of restaurant. This will invoke the first chain, which will return the menu and specialty dish. The output of this chain will be passed to the second chain, which will analyze the specialty dish for allergens.
from langchain.chains import SimpleSequentialChain
chains = SimpleSequentialChain(chains=[type_of_rest, list_alergens]) # Order of the chain matters
chains.invoke("Italian Cuisine") # only one input

{'input': 'Italian Cuisine', 'output': 'Gluten\nEggs\nMilk\nNuts\nSoy'}

### SequentialChain
SequentialChain is a more flexible type of chain that allows you to define multiple input variables for each chain in the sequence. This means you can pass different inputs to each chain, allowing for more complex workflows.

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
# type_of_rest: This instance will be used to generate a menu and specialty dish based on the cuisine type.
type_of_rest = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["cuisine_type", "dish_name"],
        template="You are a helpful assistant in a {cuisine_type} restaurant. When asked about the menu, list ingredients of the the {dish_name}",
    ),
    output_key="ingredients", # This will be used as input for the next chain (order_chain
)

# order_chain: This instance will be used to analyze the specialty dish for allergens.
order_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["dish_name", "ingredients"],
        template="You are a food quality analyst. You test food allergens and quality when asked about the speciality dish. Provide list of names of allergens into given dish. No any additional information, just list the allergens. \n Dish Name: {dish_name}, Ingredients: {ingredients}",
    ),
    output_key="allergens", # This will be used as output of the chain
)

In [11]:
from langchain.chains import SequentialChain

chains = SequentialChain(
    chains=[type_of_rest, order_chain],
    input_variables=["cuisine_type", "dish_name"],
    output_variables=["dish_name", "allergens"],
)

chains.invoke({"cuisine_type": "Italian Cuisine", "dish_name": "Margerita Pizza"})  # Example usage with multiple inputs

{'cuisine_type': 'Italian Cuisine',
 'dish_name': 'Margerita Pizza',
 'allergens': 'Milk\nWheat'}

Different type of chains are available in Langchain, checkout the [this link](https://python.langchain.com/docs/modules/chains/overview) for more details.

## Memory
Memory is a feature that allows you to store and retrieve information across multiple interactions with the LLM. It stores convertaion history.

In [12]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)
print("query 1:", conversation.invoke("You are a assistant in a Italian Restaurant. What dishes you have in the menu? just list the 5 names of the dishes without description.").get("response"))
print("query 2:", conversation.invoke("which type of cuisine do you serve? just give me the answer. No additional information.").get("response"))  # This will give Italian as an answer
print("query 3:", conversation.invoke("What 5 + 10? Just give me the answer. No additional information.").get("response"))
print("query 4:", conversation.invoke("What 11 + 5? Just give me the answer. No additional information.").get("response")) # Here, first prompt will be forgotten as we are keeping only last 1 message in the memory
print("query 5:", conversation.invoke("which type of cuisine do you serve? just give me the answer. No additional information.").get("response"))  # This will give Italian as an answer


/var/folders/vd/bybb603x54vf1kyhghxkssbc0000gn/T/ipykernel_48054/1532302122.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/var/folders/vd/bybb603x54vf1kyhghxkssbc0000gn/T/ipykernel_48054/1532302122.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm, memory=memory)


query 1: Ciao! Welcome to our trattoria! I'm so excited to tell you about our delicious offerings. We have a wonderful selection of authentic Italian dishes that I'm sure you'll love.

Let's see, from our antipasti, we have the classic **Bruschetta al Pomodoro**, which is toasted bread rubbed with garlic and topped with fresh tomatoes, basil, and olive oil. Then, for our primi piatti, we have a delightful **Spaghetti Carbonara**, made with eggs, hard cheese, cured pork, and black pepper. We also offer a rich and flavorful **Lasagne alla Bolognese**, with layers of pasta, béchamel sauce, and a hearty meat ragù.

Moving on to our secondi piatti, we have a tender **Pollo alla Cacciatora**, which is chicken braised with tomatoes, onions, herbs, and often wine. And for a taste of the sea, we have our exquisite **Gamberi all'Aglio**, which are succulent shrimp sautéed with garlic, white wine, and parsley.

So, to answer your question directly, the five dishes I've just mentioned are:

*   Br

### Limited Memory
If you want to store limited number of messages in the memory, you can use `ConversationBufferWindowMemory`. This allows you to keep a fixed number of messages in the memory, which can be useful for maintaining context without overwhelming the LLM with too much information.

In [13]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1)  # Keep last 1 message
conversation = ConversationChain(llm=llm, memory=memory)
print("query 1:", conversation.invoke("You are a assistant in a Italian Restaurant. What dishes you have in the menu? just list the 5 names of the dishes without description.").get("response"))
print("query 2:", conversation.invoke("which type of cuisine do you serve? just give me the answer. No additional information.").get("response"))  # This will give Italian as an answer
print("query 3:", conversation.invoke("What 5 + 10? Just give me the answer. No additional information.").get("response"))
print("query 4:", conversation.invoke("What 11 + 5? Just give me the answer. No additional information.").get("response")) # Here, first prompt will be forgotten as we are keeping only last 1 message in the memory
print("query 5:", conversation.invoke("which type of cuisine do you serve? just give me the answer. No additional information.").get("response"))  # This won't return Italian as an answer

/var/folders/vd/bybb603x54vf1kyhghxkssbc0000gn/T/ipykernel_48054/3667280143.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)  # Keep last 1 message


query 1: Ciao! Welcome to our little slice of Italy! I'd be absolutely delighted to tell you about some of our most popular dishes. We have a wonderful selection, but if you're looking for just five names, here are some of our absolute favorites:

1.  Spaghetti Carbonara
2.  Margherita Pizza
3.  Lasagna Bolognese
4.  Chicken Parmesan
5.  Tiramisu
query 2: Italian
query 3: 15
query 4: 16
query 5: I do not serve any type of cuisine. I am a large language model, trained by Google.


## Tools
Tools perform specifc tasks based on the input provided. Tools can be predefined or custom-built. i.e. We can use Google Search tool to fetch real-time information from the web or we can create custom tools to perform specific tasks.

### Predefined Tools

In [14]:
# Import relevant functionality
from langchain_core.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun() # Create a search tool using DuckDuckGo
tools = [search] # List of tools to be used by the agent. Used in llm to bind tools
tool_map = {tool.name: tool for tool in tools} # Useful to easily invoke the tool by name
agent = llm.bind_tools(tools) # Create an agent with the tools
resp = agent.invoke("What is the latest news about LangChain? search into Duckduckgo") # Example usage with the search tool

# Iterate through the tool calls and run the corresponding tool
for call in resp.tool_calls:
    if call.get("name") in tool_map:
        tool = tool_map[call.get("name")]
        result = tool.run(call.get("args"))
        print(f"Tool: {tool.name}, \n\nResult: {result}")

/Users/skyli997/Desktop/langchain/.venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Tool: duckduckgo_search, 

Result: Jun 17, 2025 · So the question now is: Is LangChain still needed in 2025? In this article, we’ll explore that question by comparing … Jul 8, 2025 · LangChain, an AI infrastructure startup providing tools to build and monitor LLM-powered applications, is raising a new round of … Jun 17, 2025 · Cybersecurity researchers have disclosed a now-patched security flaw in LangChain's LangSmith platform that could be … Mar 5, 2025 · LangChain launched its first course, 'Introduction to LangGraph,' through LangChain Academy, focusing on building agentic and … Jul 9, 2025 · Cofounders Harrison Chase and Ankush Gola started LangChain in 2023 as an open source software that helped engineers …


### Custom Tool

#### Example of direct tool invocation
Here. we invoke tools direclty without using LLM.

In [16]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> float:
    """
    This is a custom tool that takes two floats and returns their sum.
    """
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """
    This is a custom tool that takes two floats and returns their product.
    """
    return a * b

@tool
def divide(a: float, b: float) -> float | str:
    """
    This is a custom tool that takes two floats and returns their division.
    """
    if b == 0:
        return "Division by zero is not allowed."
    return a / b

@tool
def subtract(a: float, b: float) -> float:
    """
    This is a custom tool that takes two floats and returns their difference.
    """
    return a - b

# Direct use of tools
print("Add:", add.invoke({"a": 2, "b": 3}))
print("Multiply:", multiply.invoke({"a": 2, "b": 3}))
print("Divide by zero:", divide.invoke({"a": 2, "b": 0}))  # This will return "Division by zero is not allowed."
print("Divide:", divide.invoke({"a": 2, "b": 3}))
print("Subtract:", subtract.invoke({"a": 2, "b": 3}))

Add: 5.0
Multiply: 6.0
Divide by zero: Division by zero is not allowed.
Divide: 0.6666666666666666
Subtract: -1.0


#### Example of binding custom tools to LLM
In this example, LLM will return the tool name and input parameters to invoke the tool and we add code to invoke the tool based on the output from LLM.

In [17]:
# Bind tools to the LLM and invoke them
tools = [add, multiply, divide, subtract]
tool_map = {tool.name: tool for tool in tools}
agent = llm.bind_tools(tools)
response = agent.invoke("What is 2 + 3?")

for call in response.tool_calls:
    if call.get("name") in tool_map:
        tool = tool_map[call.get("name")]
        result = tool.run(call.get("args"))
        print(f"Tool: {tool.name}, Input: {call.get('args')}, Result: {result}")


Tool: add, Input: {'a': 2.0, 'b': 3.0}, Result: 5.0


## Agents
Agents are llm with tools which can invoke tools dynamically based on the prompt. 

In [18]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.tools import tool

# 1. Define calculator functions using @tool decorator
@tool
def add(a: float, b: float) -> float:
    """Adds two numbers."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """Subtracts b from a."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers."""
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divides a by b. Raises error if dividing by zero."""
    if b == 0:
        raise ValueError("Cannot divide by zero.")
    return a / b

# 2. Create list of tools
tools = [add, subtract, multiply, divide]

# 4. Initialize the agent with tools and LLM
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # Use structured chat agent type when tools have multiple inputs
    verbose=True
)

# 5. Run the agent with a query
agent.run("What is (5 + 3) * 2 divided by 4?")

/var/folders/vd/bybb603x54vf1kyhghxkssbc0000gn/T/ipykernel_48054/2308310953.py:32: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/var/folders/vd/bybb603x54vf1kyhghxkssbc0000gn/T/ipykernel_48054/2308310953.py:40: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is (5



> Entering new AgentExecutor chain...
I need to perform the following calculation: (5 + 3) * 2 / 4.
First, I will calculate the sum inside the parentheses.
Then, I will multiply the result by 2.
Finally, I will divide the product by 4.
I will use the `add` tool for the first step.
Action:
```
{
  "action": "add",
  "action_input": {
    "a": 5,
    "b": 3
  }
}
```
Observation: 8.0
Thought:The result of 5 + 3 is 8. Now I need to multiply this result by 2. I will use the `multiply` tool for this step.
Action:
```
{
  "action": "multiply",
  "action_input": {
    "a": 8.0,
    "b": 2
  }
}
```
Observation: 16.0
Thought:The result of (5 + 3) * 2 is 16. Now I need to divide this result by 4. I will use the `divide` tool for this step.
Action:
```
{
  "action": "divide",
  "action_input": {
    "a": 16.0,
    "b": 4
  }
}
```
Observation: 4.0
Thought:The result of the calculation (5 + 3) * 2 / 4 is 4.0. I have completed all the steps.
Action:
```
{
  "action": "Final Answer",
  "action_in

'4.0'